# Introduction to Recurrent Neural Networks

In [1]:
using Knet

## A one layer MLP vs a simple RNN

([Elman 1990](http://onlinelibrary.wiley.com/doi/10.1207/s15516709cog1402_1/pdf)) A simple RNN takes the previous hidden state as an extra input, and returns the next hidden state as an output.

In [2]:
# Comparison of a single MLP layer and corresponding RNN

function tanhmlp(param, input)
    weight,bias = param
    return tanh(input * weight .+ bias)
end

function tanhrnn(param, input, state)
    weight,bias = param
    return tanh(hcat(input, state) * weight .+ bias)
end;

<img src="images/rnn-vs-mlp.png" />(<a href="https://docs.google.com/drawings/d/1bPttFA0GEh7ti3xoWDma1ZbrQ21eulsPDeVUbPgdA7g/edit?usp=sharing">image source</a>)

## Architectures

<img src="images/diags.png" />
(<a href="http://karpathy.github.io/2015/05/21/rnn-effectiveness">image source</a>)

## Long Short-Term Memory (LSTM)
([Hochreiter and Schmidhuber, 1997](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf))
LSTM is a more sophisticated RNN module that performs better with long-range dependencies. 

<img src="images/LSTM3-chain.png" width=800 />
([image source](http://colah.github.io/posts/2015-08-Understanding-LSTMs))



$$\begin{align}
f_t &= \sigma(W_f\cdot[h_{t-1},x_t] + b_f) & \text{forget gate} \\
i_t &= \sigma(W_i\cdot[h_{t-1},x_t] + b_i) & \text{input gate} \\
\tilde{C}_t &= \tanh(W_C\cdot[h_{t-1},x_t] + b_C) & \text{cell candidate} \\
C_t &= f_t \ast C_{t-1} + i_t \ast \tilde{C}_t & \text{new cell} \\
o_t &= \sigma(W_o\cdot[h_{t-1},x_t] + b_o) & \text{output gate} \\
h_t &= o_t \ast \tanh(C_t) & \text{new output}\\
\end{align}$$

## A pure Julia LSTM implementation

In [3]:
function lstm(param, input, state)
    weight,bias = param
    hidden,cell = state
    h       = size(hidden,2)
    gates   = hcat(input,hidden) * weight .+ bias
    forget  = sigm.(gates[:,1:h])
    ingate  = sigm.(gates[:,1+h:2h])
    outgate = sigm.(gates[:,1+2h:3h])
    change  = tanh.(gates[:,1+3h:4h])
    cell    = cell .* forget + ingate .* change
    hidden  = outgate .* tanh.(cell)
    return (hidden,cell)
end;

## Gated Recurrent Unit (GRU)

Introduced by <a href="http://arxiv.org/pdf/1406.1078v3.pdf">Cho, et al. (2014)</a>, GRU combines the forget and input gates into a single “update gate.”

<img src="images/LSTM3-var-GRU.png" />
(<a href="http://colah.github.io/posts/2015-08-Understanding-LSTMs/">image source</a>)



## A more efficient cuDNN based implementation: Knet.RNN

In [3]:
using Knet: RNN
@doc RNN

┌ Info: Precompiling Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1186


```
rnn = RNN(inputSize, hiddenSize; opts...)
rnn(x, h...; hout, batchSizes) => y
```

`RNN` returns a callable RNN object `rnn`. Given a minibatch of sequences `x`, `rnn(x)` returns `y`, the hidden states of the final layer for each time step.

**Dimensions:** The input `x` can be 1, 2, or 3 dimensional and `y` will have the same number of dimensions as `x`. size(x)=(X,[B,T]) and size(y)=(H/2H,[B,T]) where X is inputSize, H is hiddenSize, B is batchSize, T is seqLength. By default a 1-D `x` represents a single instance for a single time step, a 2-D `x` represents a single minibatch for a single time step, and a 3-D `x` represents a sequence of identically sized minibatches for multiple time steps. The output `y` gives the hidden state (of the final layer for multi-layer RNNs) for each time step, its first dimension is H for unidirectional and 2H for bidirectional RNNs.

**batchSizes:** If `batchSizes=nothing` (default), all sequences in a minibatch are assumed to be the same length. If `batchSizes` is an array of (non-increasing) integers, it gives us the batch size for each time step (allowing different sequences in the minibatch to have different lengths). In this case `x` will typically be 2-D with the second dimension representing variable size batches for time steps. If `batchSizes` is used, `sum(batchSizes)` should equal `length(x) ÷ size(x,1)`.

**Hidden states:** Additional arguments to rnn, `h...`, can optionally be used to specify the initial hidden and cell states (cell state is only used by LSTMs). Initial states are assumed to be zero if not specified.  Final hidden states (for all layers) will be discarded if `hout=nothing` (default), or pushed into `hout` (which should be Any[]) otherwise. All hidden and cell states have dimensionality (H,B,L) for unidirectional and (H,B,2L) for bidirectional RNNs.  If `batchSizes` is used and minibatch sizes change over time, B is always taken to be the size of the first minibatch for hidden sizes.

**Keyword arguments for RNN:**

  * `rnnType=:lstm` Type of RNN: One of :relu, :tanh, :lstm, :gru.
  * `numLayers=1`: Number of RNN layers.
  * `bidirectional=false`: Create a bidirectional RNN if `true`.
  * `dropout=0`: Dropout probability. Ignored if `numLayers==1`.
  * `skipInput=false`: Do not multiply the input with a matrix if `true`.
  * `dataType=Float32`: Data type to use for weights.
  * `algo=0`: Algorithm to use, see CUDNN docs for details.
  * `seed=0`: Random number seed for dropout. Uses `time()` if 0.
  * `winit=xavier`: Weight initialization method for matrices.
  * `binit=zeros`: Weight initialization method for bias vectors.
  * `usegpu=(gpu()>=0)`: GPU used by default if one exists.

**Formulas:** RNNs compute the output h[t] for a given iteration from the recurrent input h[t-1] and the previous layer input x[t] given matrices W, R and biases bW, bR from the following equations:

`:relu` and `:tanh`: Single gate RNN with activation function f:

```
h[t] = f(W * x[t] .+ R * h[t-1] .+ bW .+ bR)
```

`:gru`: Gated recurrent unit:

```
i[t] = sigm(Wi * x[t] .+ Ri * h[t-1] .+ bWi .+ bRi) # input gate
r[t] = sigm(Wr * x[t] .+ Rr * h[t-1] .+ bWr .+ bRr) # reset gate
n[t] = tanh(Wn * x[t] .+ r[t] .* (Rn * h[t-1] .+ bRn) .+ bWn) # new gate
h[t] = (1 - i[t]) .* n[t] .+ i[t] .* h[t-1]
```

`:lstm`: Long short term memory unit with no peephole connections:

```
i[t] = sigm(Wi * x[t] .+ Ri * h[t-1] .+ bWi .+ bRi) # input gate
f[t] = sigm(Wf * x[t] .+ Rf * h[t-1] .+ bWf .+ bRf) # forget gate
o[t] = sigm(Wo * x[t] .+ Ro * h[t-1] .+ bWo .+ bRo) # output gate
n[t] = tanh(Wn * x[t] .+ Rn * h[t-1] .+ bWn .+ bRn) # new gate
c[t] = f[t] .* c[t-1] .+ i[t] .* n[t]               # cell output
h[t] = o[t] .* tanh(c[t])
```


In [ ]:
# Download data
